In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision
from google.protobuf import text_format
import onnx
import numpy as np
import os
from onnx_tf.backend import run_model
from matplotlib import pyplot as plt

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=2)
        self.fc1 = nn.Linear(720, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 720)
        x = F.relu(self.fc1(x))
        return F.softmax(x, dim=1)

In [13]:
model = Net()
dummy_input = Variable(torch.randn(1, 3, 32, 32))
torch.onnx.export(model, dummy_input, "pb/torch_gh.pb", verbose=True, export_params=True)

graph(%0 : Float(1, 3, 32, 32)
      %1 : Float(10, 3, 3, 3)
      %2 : Float(10)
      %3 : Float(20, 10, 5, 5)
      %4 : Float(20)
      %5 : Float(10, 720)
      %6 : Float(10)) {
  %7 : Float(1, 10, 15, 15) = Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[2, 2]](%0, %1, %2), scope: Net/Conv2d[conv1]
  %8 : Float(1, 10, 15, 15) = Relu(%7), scope: Net
  %9 : Float(1, 20, 6, 6) = Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[2, 2]](%8, %3, %4), scope: Net/Conv2d[conv2]
  %10 : Float(1, 20, 6, 6) = Relu(%9), scope: Net
  %11 : Float(1, 720) = Reshape[shape=[-1, 720]](%10), scope: Net
  %12 : Float(1, 10) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%11, %5, %6), scope: Net/Linear[fc1]
  %13 : Float(1, 10) = Relu(%12), scope: Net
  %14 : Float(1, 10) = Softmax[axis=1](%13), scope: Net
  return (%14);
}



In [45]:
data = np.random.randn(1, 3, 32, 32).astype(np.float32)

In [48]:
input = Variable(torch.from_numpy(data))
model(input)


 0.0217  0.0217  0.0217  0.1278  0.3326  0.0217  0.3712  0.0217  0.0217  0.0379
[torch.FloatTensor of size (1,10)]

In [49]:
with open(os.path.join('pb/torch_gh.pb'), "rb") as f:
    test = onnx.load(f)
    # data = np.transpose(data, (0, 2, 3, 1))
    rs_onnx = run_model(test, [data])
    print(rs_onnx)

/Users/wenhao/Projects/onnx-tensorflow/onnx_tf/backend.py:708: UserWarning: Unsupported kernel_shape attribute by Tensorflow in Conv operator. The attribute will be ignored.
  UserWarning)


Outputs(_0=array([[0.02174817, 0.02174817, 0.02174817, 0.1278409 , 0.33256865,
        0.02174817, 0.37121472, 0.02174817, 0.02174817, 0.03788668]],
      dtype=float32))
